In [ ]:
!pip install SimpleITK

In [20]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import SimpleITK as sitk
import plotly.express as px
import plotly.graph_objects as go
import math

In [3]:
def load_img_from_tiff(path2img):
    img = sitk.ReadImage(path2img)
    img_array = sitk.GetArrayFromImage(img)
    return(img_array)

In [ ]:
path = "/content/drive/MyDrive/mydata/sea_urchin_data/2D/11C1 top view.tif"
image_numpy=load_img_from_tiff(path)
# px.imshow(255-image_numpy, color_continuous_scale='gray')
px.imshow(image_numpy)

In [5]:
# adaptive threshold?
ad_thresh=cv2.adaptiveThreshold(image_numpy,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,41,2)
#50 bin thresh work better??
# px.imshow(ad_thresh, color_continuous_scale='gray')

In [ ]:
contours, hierarcy = cv2.findContours(255-ad_thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
drawn_contours=cv2.drawContours(np.zeros_like(ad_thresh), contours, -1, (255), 1)
px.imshow(drawn_contours, color_continuous_scale='gray')

In [ ]:
comparision=cv2.drawContours(image_numpy.copy(), contours, -1, (255), 1)
px.imshow(comparision, color_continuous_scale='gray')

In [ ]:
blank=np.zeros_like(ad_thresh)
for cnt in contours:

    area = cv2.contourArea(cnt)
    if area > 10:

        M = cv2.moments(cnt)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        centroids_and_conts=cv2.circle(comparision, (cX, cY), 1, (255, 255, 255),1)
        centroids = cv2.circle(blank, (cX, cY), 1, (255, 255, 255),1)

px.imshow(centroids_and_conts, color_continuous_scale='gray')

In [ ]:
px.imshow(centroids, color_continuous_scale='gray')

In [21]:
perimeters=[]
areas=[]
for cnt in contours:
    peri = cv2.arcLength(cnt, True)
    area = cv2.contourArea(cnt)
    perimeters.append(peri)
    areas.append(area)

area_circle_with_same_perimeter=[(i*i)/(4*math.pi) for i in perimeters]
areas, area_circle_with_same_perimeter = zip(*sorted(zip(areas, area_circle_with_same_perimeter)))


In [ ]:
import plotly.graph_objects as go
idx=[i for i in range(len(areas)//4)]

fig = go.Figure(data=[
    go.Bar(name='Area of polygon', x=idx, y=areas),
    go.Bar(name='Area of circle of equivalent perimeter', x=idx, y=area_circle_with_same_perimeter)
])
# Change the bar mode
fig.update_layout(title_text="Comparison - Areas of Pores Vs Areas of Circle of Equivalent Perimeter", 
                  barmode='group', 
                  bargap=0.40, # gap between bars of adjacent location coordinates.
                  bargroupgap=0.0) # gap between bars of the same location coordinate.
fig.show()

In [10]:
# radius
# node map
# area
# area hole heatmap
#